# Simple LeNet for MNIST classification

Inspired by Yann LeCunns LeNet-5 (Y. LeCun, L. Bottou, Y. Bengio, and P. Haffner. Gradient-based learning applied to document recognition. Proceedings of the IEEE, 1998).

In [1]:
using Knet
using NNHelferlein
using MLDatasets: MNIST

┌ Info: Precompiling NNHelferlein [b9e938e5-d80d-48a2-bb0e-6649b4a98aeb]
└ @ Base loading.jl:1423


### Get MNIST data from MLDatasets:

In [2]:
mnist_dir = joinpath(NNHelferlein.DATA_DIR, "mnist")
xtrn,ytrn = MNIST.traindata(Float32, dir=mnist_dir)
ytrn[ytrn.==0] .= 10
dtrn = minibatch(xtrn, ytrn, 128; xsize = (28,28,1,:))

xvld,yvld = MNIST.testdata(Float32, dir=mnist_dir)
yvld[yvld.==0] .= 10
dvld = minibatch(xvld, yvld, 128; xsize = (28,28,1,:));

### Define LeNet with NNHelferlein types:

In [3]:
lenet = Classifier(Conv(5,5,1,20),       
                Pool(),
                Conv(5,5,20,50),
                Pool(),
                Flat(),
                Dense(800,512), 
                Dense(512,10, actf=identity)
        )

Classifier((Conv(P(KnetArray{Float32, 4}(5,5,1,20)), P(KnetArray{Float32, 4}(1,1,20,1)), Knet.Ops20.relu, Base.Pairs{Symbol, Union{}, Tuple{}, NamedTuple{(), Tuple{}}}()), Pool(Base.Pairs{Symbol, Union{}, Tuple{}, NamedTuple{(), Tuple{}}}()), Conv(P(KnetArray{Float32, 4}(5,5,20,50)), P(KnetArray{Float32, 4}(1,1,50,1)), Knet.Ops20.relu, Base.Pairs{Symbol, Union{}, Tuple{}, NamedTuple{(), Tuple{}}}()), Pool(Base.Pairs{Symbol, Union{}, Tuple{}, NamedTuple{(), Tuple{}}}()), Flat(), Dense(P(Knet.KnetArrays.KnetMatrix{Float32}(512,800)), P(Knet.KnetArrays.KnetVector{Float32}(512)), Knet.Ops20.sigm), Dense(P(Knet.KnetArrays.KnetMatrix{Float32}(10,512)), P(Knet.KnetArrays.KnetVector{Float32}(10)), identity)))

### Train with Tensorboard logger:

In [4]:
tb_train!(lenet, Adam, dtrn, dvld, epochs=2,
        acc_fun=accuracy,
        eval_size=0.2, eval_freq=5, mb_loss_freq=100, 
        tb_name="example_run", tb_text="NNHelferlein example")

Training 2 epochs with 468 minibatches/epoch
    (and 78 validation mbs).
Evaluation is performed every 94 minibatches (with 16 mbs).
Watch the progress with TensorBoard at: /home/andreas/.julia/dev/NNHelferlein/examples/logs/example_run/2021-12-22T10-33-28


Progress: 100%|█████████████████████████████████████████| Time: 0:00:14


Training finished with:
Training loss:       0.04167936032271793
Training accuracy:   0.9879139957264957
Validation loss:     0.042134712426326215
Validation accuracy: 0.9856770833333334


Classifier((Conv(P(KnetArray{Float32, 4}(5,5,1,20)), P(KnetArray{Float32, 4}(1,1,20,1)), Knet.Ops20.relu, Base.Pairs{Symbol, Union{}, Tuple{}, NamedTuple{(), Tuple{}}}()), Pool(Base.Pairs{Symbol, Union{}, Tuple{}, NamedTuple{(), Tuple{}}}()), Conv(P(KnetArray{Float32, 4}(5,5,20,50)), P(KnetArray{Float32, 4}(1,1,50,1)), Knet.Ops20.relu, Base.Pairs{Symbol, Union{}, Tuple{}, NamedTuple{(), Tuple{}}}()), Pool(Base.Pairs{Symbol, Union{}, Tuple{}, NamedTuple{(), Tuple{}}}()), Flat(), Dense(P(Knet.KnetArrays.KnetMatrix{Float32}(512,800)), P(Knet.KnetArrays.KnetVector{Float32}(512)), Knet.Ops20.sigm), Dense(P(Knet.KnetArrays.KnetMatrix{Float32}(10,512)), P(Knet.KnetArrays.KnetVector{Float32}(10)), identity)))

Tensorboard output:    
<img src="assets/10-mnist_tb.png">